# 1D noiseless illustrative example


In [19]:
# import standard libraries
from mfbml.problems import mf_Forrester
from mfbml.methods import KernelRidgeLinearGaussianProcess, KernelRidgeRegression
from mfbml.metrics.accuracy_metrics import normalized_mae, normalized_rmse
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings

warnings.filterwarnings("ignore")

In [20]:
# fix the random seed for reproducibility
seed_number = 1
np.random.seed(seed_number)

## Get function and design of experiment


In [21]:
# define function
func = mf_Forrester()
design_space = func.design_space

# update sample_x['hf']
sample_xh = np.array([0.0, 0.4, 0.6, 1.0]).reshape((-1, 1))
sample_xl = np.linspace(0, 1, 11, endpoint=True).reshape((-1, 1))
sample_x = [sample_xh, sample_xl]
# get response
sample_yh = func.hf(sample_x[0])
sample_yl = func.lf_2(sample_x[1])
sample_y = [sample_yh, sample_yl]
# generate test samples
test_x = np.linspace(0, 1, 1000).reshape(-1, 1)
test_hy = func.hf(test_x)
test_ly = func.lf_2(test_x)

### MF-RBF-Krging with linear lf surrogate


In [22]:

linear_model = KernelRidgeLinearGaussianProcess(
    design_space=func.design_space,
    optimizer_restart=20,
    noise_prior=0.0,
    seed=seed_number
)
linear_model.lf_model = KernelRidgeRegression(
    design_space=func.design_space, params_optimize=False, noise_data=False,
)
linear_model.train(X=sample_x,
                   Y=sample_y)
linear_pred_y, linear_pred_std = linear_model.predict(
    X=test_x, return_std=True)

# get prediction of low fidelity
pred_ly = linear_model.predict_lf(X=test_x)
end_time = time.time()


# calculate accuracy metrics
nrmse_linear = normalized_rmse(test_hy, linear_pred_y)
nmae_linear = normalized_mae(test_hy, linear_pred_y)
r2_linear = r2_score(test_hy, linear_pred_y)
cpu_time_linear = linear_model.lf_training_time + linear_model.hf_training_time
print("nrmse: ", nrmse_linear)
print("nmae: ", nmae_linear)
print("r2: ", r2_linear)
print("cpu_time: ", cpu_time_linear)

nrmse:  0.007260479255726432
nmae:  0.017530855041700887
r2:  0.9999822268894117
cpu_time:  0.05485248565673828


## MF-RBF-Krging with linear lf surrogate(without bias)


In [23]:
# train model
start_time = time.time()
linear_without_const_model = KernelRidgeLinearGaussianProcess(
    design_space=func.design_space,
    lf_poly_order="linear_without_const",
    noise_prior=0.0,
    optimizer_restart=20,
)
linear_without_const_model.lf_model = KernelRidgeRegression(
    design_space=func.design_space, params_optimize=False, noise_data=False,
)
linear_without_const_model.train(X=sample_x,
                                 Y=sample_y)
# get prediction of low fidelity
linear_without_const_pred_ly = linear_without_const_model.predict_lf(X=test_x)
# get prediction of high fidelity
linear_without_const_pred_hy, linear_without_const_pred_std = (
    linear_without_const_model.predict(X=test_x, return_std=True)
)
end_time = time.time()
# calculate accuracy metrics
nrmse_linear_without_const = normalized_rmse(
    test_hy, linear_without_const_pred_hy)
nmae_linear_without_const = normalized_mae(
    test_hy, linear_without_const_pred_hy)
r2_linear_without_const = r2_score(test_hy, linear_without_const_pred_hy)
cpu_time_linear_without_const = end_time - start_time

# print accuracy metrics
print("normalized rmse: ", nrmse_linear_without_const)
print("normalized mae: ", nmae_linear_without_const)
print("r2: ", r2_linear_without_const)
print("cpu_time: ", cpu_time_linear_without_const)

normalized rmse:  0.007246057149893062
normalized mae:  0.017524770503425515
r2:  0.9999822974277583
cpu_time:  0.07048463821411133


In [24]:
# quadratic model

quadratic_model = KernelRidgeLinearGaussianProcess(
    design_space=func.design_space, lf_poly_order="quadratic", optimizer_restart=20, noise_prior=0.0
)
quadratic_model.lf_model = KernelRidgeRegression(design_space=func.design_space, noise_data=False,
                                                 params_optimize=False)
quadratic_model.train(X=sample_x, Y=sample_y)
# get prediction of low fidelity
quadratic_pred_ly = quadratic_model.predict_lf(X=test_x)
# get prediction of high fidelity
quadratic_pred_hy, quadratic_pred_std = quadratic_model.predict(
    X=test_x, return_std=True
)


# calculate accuracy metrics
nrmse_quadratic = normalized_rmse(test_hy, quadratic_pred_hy)
nmae_quadratic = normalized_mae(test_hy, quadratic_pred_hy)
r2_quadratic = r2_score(test_hy, quadratic_pred_hy)
cpu_time_quadratic = quadratic_model.lf_training_time + \
    quadratic_model.hf_training_time

# print accuracy metrics
print("normalized rmse: ", nrmse_quadratic)
print("normalized mae: ", nmae_quadratic)
print("r2: ", r2_quadratic)
print("cpu_time: ", cpu_time_quadratic)

normalized rmse:  1.132083543869476
normalized mae:  2.373799733843675
r2:  0.5678944740524126
cpu_time:  0.12982845306396484


In [25]:
# cubic model
cubic_model = KernelRidgeLinearGaussianProcess(
    design_space=func.design_space, lf_poly_order="cubic", optimizer_restart=20
)
cubic_model.lf_model = KernelRidgeRegression(design_space=func.design_space,
                                             noise_data=False,
                                             params_optimize=False)
cubic_model.train(X=sample_x, Y=sample_y)
# get prediction of low fidelity
cubic_pred_ly = cubic_model.predict_lf(X=test_x)
# get prediction of high fidelity
cubic_pred_hy, cubic_pred_std = cubic_model.predict(X=test_x, return_std=True)
end_time = time.time()

# calculate accuracy metrics
nrmse_cubic = normalized_rmse(test_hy, cubic_pred_hy)
nmae_cubic = normalized_mae(test_hy, cubic_pred_hy)
r2_cubic = r2_score(test_hy, cubic_pred_hy)
cpu_time_cubic = end_time - start_time

# print accuracy metrics
print("normalized mse: ", nrmse_cubic)
print("normalized mae: ", nmae_cubic)
print("r2: ", r2_cubic)
print("cpu_time: ", cpu_time_cubic)

normalized mse:  1.1262889098941213
normalized mae:  2.285151882785765
r2:  0.5723066664473351
cpu_time:  0.37488698959350586


In [26]:
# output the beta values for each model
print("beta values for linear model: ", linear_model.beta)
print(
    "beta values for linear without constant model: ", linear_without_const_model.beta
)
print("beta values for quadratic model: ", quadratic_model.beta)
print("beta values for scaled kriging model: ", cubic_model.beta)

beta values for linear model:  [[2.91830941]
 [1.99323047]]
beta values for linear without constant model:  [[1.99326057]]
beta values for quadratic model:  [[0.48952908]
 [2.06122326]
 [0.83458136]]
beta values for scaled kriging model:  [[ 0.92691295]
 [ 1.69552142]
 [-0.08732022]
 [-0.31596954]]


In [27]:
# plot results in 2x2 grid
font_size = 16
plt.rcParams["text.usetex"] = True
fig, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[1].plot(
    sample_x[0],
    sample_y[0],
    "kx",
    linewidth=2,
    markersize=10,
    label="HF samples",
)
ax[1].plot(
    sample_x[1],
    sample_y[1],
    "+",
    color="#AA4499",
    linewidth=2,
    markersize=10,
    alpha=0.5,
    label="LF samples",
)
ax[1].plot(test_x, test_hy, "--", color="#33BBEE",
           linewidth=2, label="HF truth")
ax[1].plot(
    test_x, linear_pred_y, "-", color="#CC3311", linewidth=2, label="HF prediction"
)
ax[1].plot(
    test_x,
    np.dot(linear_model._basis_function(test_x), linear_model.beta)
    * linear_model.yh_std
    + linear_model.yh_mean,
    "-",
    color="#AA4499",
    linewidth=2,
    label="fitted basis function",
)
ax[1].plot(test_x, test_ly, "--", color="#EE7733",
           linewidth=2, label="LF truth")
ax[1].plot(test_x, pred_ly, "-", color="#0077BB",
           linewidth=2, label="LF prediction")

ax[1].fill_between(
    test_x.flatten(),
    (linear_pred_y - 2 * linear_pred_std).flatten(),
    (linear_pred_y + 2 * linear_pred_std).flatten(),
    alpha=0.5,
    color="#BBBBBB",
    label="CI interval",
)
ax[1].set_title("Linear", fontsize=font_size)
ax[1].set_ylabel(" ", fontsize=font_size)
ax[1].set_xlabel("x", fontsize=font_size)
# ax[1].legend()
# for hierarchical kriging
ax[2].plot(
    sample_x[0],
    sample_y[0],
    "kx",
    linewidth=2,
    markersize=10,
    label="HF samples",
)
ax[2].plot(
    sample_x[1],
    sample_y[1],
    "+",
    color="#AA4499",
    linewidth=2,
    markersize=10,
    alpha=0.5,
    label="LF samples",
)
ax[2].plot(test_x, test_hy, "--", color="#33BBEE",
           linewidth=2, label="HF truth")
ax[2].plot(
    test_x, quadratic_pred_hy, "-", color="#CC3311", linewidth=2, label="HF prediction"
)
ax[2].plot(test_x, test_ly, "--", color="#EE7733",
           linewidth=2, label="LF truth")
ax[2].plot(
    test_x, quadratic_pred_ly, "-", color="#0077BB", linewidth=2, label="LF prediction"
)

ax[2].plot(
    test_x,
    np.dot(
        quadratic_model._basis_function(test_x, poly_order="quadratic"),
        quadratic_model.beta,
    )
    * quadratic_model.yh_std
    + quadratic_model.yh_mean,
    "-",
    color="#AA4499",
    linewidth=2,
    label="fitted basis function",
)
ax[2].fill_between(
    test_x.flatten(),
    (quadratic_pred_hy - 2 * quadratic_pred_std).flatten(),
    (quadratic_pred_hy + 2 * quadratic_pred_std).flatten(),
    alpha=0.5,
    color="#BBBBBB",
    label="CI interval",
)
ax[2].set_title("Quadratic", fontsize=font_size)
ax[2].set_ylabel(" ", fontsize=font_size)
ax[2].set_xlabel("x", fontsize=font_size)

# for co-kriging
ax[3].plot(
    sample_x[0],
    sample_y[0],
    "kx",
    linewidth=2,
    markersize=10,
    label="HF samples",
)
ax[3].plot(test_x, test_hy, "--", color="#33BBEE",
           linewidth=2, label="HF truth")
ax[3].plot(
    test_x, cubic_pred_hy, "-", color="#CC3311", linewidth=2, label="HF prediction"
)

ax[3].plot(test_x, test_ly, "--", color="#EE7733",
           linewidth=2, label="LF truth")
ax[3].plot(
    test_x, cubic_pred_ly, "-", color="#0077BB", linewidth=2, label="LF prediction"
)
ax[3].plot(
    sample_x[1],
    sample_y[1],
    "+",
    color="#AA4499",
    linewidth=2,
    markersize=10,
    label="LF samples",
)
ax[3].plot(
    test_x,
    np.dot(cubic_model._basis_function(
        test_x, poly_order="cubic"), cubic_model.beta)
    * cubic_model.yh_std
    + cubic_model.yh_mean,
    "-",
    color="#AA4499",
    linewidth=2,
    label=r"$\mathbf{m}(x)^T\rho$",
)
ax[3].fill_between(
    test_x.flatten(),
    (cubic_pred_hy - 2 * cubic_pred_std).flatten(),
    (cubic_pred_hy + 2 * cubic_pred_std).flatten(),
    alpha=0.5,
    color="#BBBBBB",
    label="CI interval",
)
ax[3].set_title("Cubic", fontsize=font_size)
ax[3].set_ylabel(" ", fontsize=font_size)
ax[3].set_xlabel("x", fontsize=font_size)

# for scaled kriging
ax[0].plot(
    sample_x[0],
    sample_y[0],
    "kx",
    linewidth=2,
    markersize=10,
    label="HF samples",
)

ax[0].plot(
    sample_x[1],
    sample_y[1],
    "+",
    color="#AA4499",
    linewidth=2,
    markersize=10,
    label="LF samples",
)
ax[0].plot(test_x, test_hy, "--", color="#33BBEE",
           linewidth=2, label="HF truth")
ax[0].plot(
    test_x,
    linear_without_const_pred_hy,
    "-",
    color="#CC3311",
    linewidth=2,
    label="HF prediction",
)
ax[0].plot(test_x, test_ly, "--", color="#EE7733",
           linewidth=2, label="LF truth")
ax[0].plot(
    test_x,
    linear_without_const_pred_ly,
    "-",
    color="#0077BB",
    linewidth=2,
    label="LF prediction",
)

ax[0].plot(
    test_x,
    np.dot(
        linear_without_const_model._basis_function(
            test_x, poly_order="linear_without_const"
        ),
        linear_without_const_model.beta,
    )
    * linear_without_const_model.yh_std
    + linear_without_const_model.yh_mean,
    "-",
    color="#AA4499",
    linewidth=2,
    label=r"$\mathbf{m}(\mathbf{x})^T\boldsymbol{\rho}$",
)
ax[0].fill_between(
    test_x.flatten(),
    (linear_without_const_pred_hy - 2 * linear_without_const_pred_std).flatten(),
    (linear_without_const_pred_hy + 2 * linear_without_const_pred_std).flatten(),
    alpha=0.5,
    color="#BBBBBB",
    label="CI interval",
)

ax[0].set_title("Linear without bias", fontsize=font_size)
ax[0].set_ylabel("y", fontsize=font_size)
ax[0].set_xlabel("x", fontsize=font_size)

#
ax[0].text(
    0.5,
    15.0,
    r"$\hat \rho_1$ = {:.2f}".format(
        linear_without_const_model.beta[0, 0],
    ),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)
ax[1].text(
    0.5,
    28.0,
    r"$\hat \rho_0$ = {:.2f}".format(linear_model.beta[0, 0]),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)
ax[1].text(
    0.5,
    24.0,
    r"$\hat \rho_1$ = {:.2f}".format(linear_model.beta[1, 0]),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)

ax[2].text(
    0.5,
    15.0,
    r"$\hat \rho_0$ = {:.2f}".format(
        quadratic_model.beta[0, 0],
    ),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)
ax[2].text(
    0.5,
    12.0,
    r"$\hat \rho_1$ = {:.2f}".format(
        quadratic_model.beta[1, 0],
    ),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)
ax[2].text(
    0.5,
    9.0,
    r"$\hat \rho_2$ = {:.2f}".format(
        quadratic_model.beta[2, 0],
    ),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)
ax[3].text(
    0.5,
    15.0,
    r"$\hat \rho_0$ = {:.2f}".format(
        cubic_model.beta[0, 0],
    ),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)
ax[3].text(
    0.5,
    12.0,
    r"$\hat \rho_1$ = {:.2f}".format(
        cubic_model.beta[1, 0],
    ),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)
ax[3].text(
    0.5,
    9.0,
    r"$\hat \rho_2$ = {:.2f}".format(
        cubic_model.beta[2, 0],
    ),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)
ax[3].text(
    0.5,
    6.0,
    r"$\hat \rho_3$ = {:.2f}".format(
        cubic_model.beta[3, 0],
    ),
    fontsize=font_size,
    ha="center",
    va="center",
    backgroundcolor="white",
)

plt.legend(loc="upper center", bbox_to_anchor=(-1.4, 1.25), ncol=8, fontsize=12)

# set the font size of axis
for i in range(3):
    ax[i].tick_params(axis="both", which="major", labelsize=font_size)
    # set the line width of the axis
    for axis in ["top", "bottom", "left", "right"]:
        ax[i].spines[axis].set_linewidth(1.5)
plt.savefig("different_basis_order_comparison.pdf",
            bbox_inches="tight", dpi=300)
plt.savefig("different_basis_order_comparison.png",
            bbox_inches="tight", dpi=300)
plt.savefig("different_basis_order_comparison.svg",
            bbox_inches="tight", dpi=300)
plt.show()

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpksveu2k1 9f558406edbc51514263636e58d51433.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./9f558406edbc51514263636e58d51433.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages

Error in callback <function _draw_all_if_interactive at 0x761bb93cb910> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpgonnqgmw 9f558406edbc51514263636e58d51433.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./9f558406edbc51514263636e58d51433.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages

RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpv3q54j3e 9f558406edbc51514263636e58d51433.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./9f558406edbc51514263636e58d51433.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/type1cm/type1cm.sty)

! LaTeX Error: File `type1ec.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.8 \usepackage
               [utf8]{inputenc}^^M
No pages

<Figure size 1600x400 with 4 Axes>

In [ ]:
# save the results into a pandas dataframe
df = pd.DataFrame(columns=["nrmse", "nmae", "r2",
                  "lf_training_time", "hf_training_time"])
df.loc["linear_without_bias"] = [
    nrmse_linear_without_const,
    nmae_linear_without_const,
    r2_linear_without_const,
    linear_without_const_model.lf_training_time,
    linear_without_const_model.hf_training_time,
]
df.loc["linear"] = [nrmse_linear, nmae_linear, r2_linear,
                    linear_model.lf_training_time, linear_model.hf_training_time]
df.loc["quadratic"] = [
    nrmse_quadratic,
    nmae_quadratic,
    r2_quadratic,
    quadratic_model.lf_training_time,
    quadratic_model.hf_training_time,
]
df.loc["cubic"] = [
    nrmse_cubic,
    nmae_cubic,
    r2_cubic,
    cubic_model.lf_training_time,
    cubic_model.hf_training_time
]
# save the results into a csv file
df.to_csv("result_different_lf_basis_order.csv")

# print the results
print(df)

                        nrmse      nmae        r2  lf_training_time  \
linear_without_bias  0.007246  0.017525  0.999982          0.001266   
linear               0.007260  0.017531  0.999982          0.006524   
quadratic            1.132084  2.373800  0.567894          0.003323   
cubic                1.126289  2.285152  0.572307          0.010998   

                     hf_training_time  
linear_without_bias          0.069910  
linear                       0.055981  
quadratic                    0.099226  
cubic                        0.176503  
